In [4]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline
import os
import numpy as np

In [5]:
fichero = "C:/Users/Marina/Documents/Marina/Universidad/Máster/Curso 1/Cuatrimestre 1/Tratamiento de Datos/Proyecto_datos/full_format_recipes.json"
data = pd.read_json(fichero)
data.replace(r'^\s*$', np.nan, regex=True, inplace=True)  # Reemplazar cadenas vacías por NaN
data_clean = data.dropna()
data_numeric = data_clean.select_dtypes(include=['number'])

# Copia del DataFrame original para aplicar la limpieza
data_cleaned = data_clean.copy()

exclude_column = 'rating'
# Iterar columna por columna y eliminar outliers usando IQR
for column in data_numeric.columns:
    if column == exclude_column:  # Saltar la columna 'fat'
        continue
    Q1 = data_cleaned[column].quantile(0.25)  # Primer cuartil
    Q3 = data_cleaned[column].quantile(0.75)  # Tercer cuartil
    IQR = Q3 - Q1                             # Rango intercuartílico

    # Definir los límites
    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 2 * IQR

    # Filtrar filas dentro de los límites para la columna actual
    data_cleaned = data_cleaned[
        (data_cleaned[column] >= lower_bound) & (data_cleaned[column] <= upper_bound)
    ]

In [ ]:
data_cleaned['texto_entrada'] = "Ingredientes: " + data_cleaned['ingredients'].astype(str)
data_cleaned['texto_salida'] = "Título: " + data_cleaned['title'].astype(str)

train_data = data_cleaned[['texto_entrada', 'texto_salida']]
train_data_formatted = train_data.apply(lambda row: f"{row['texto_entrada']} {row['texto_salida']}", axis=1)

train_file = "titulos_generados.txt"
train_data_formatted.to_csv(train_file, index=False, header=False)

def load_dataset(file_path, tokenizer, block_size=512):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

dataset = load_dataset(train_file, tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,  
    per_device_train_batch_size=8,  
    save_steps=100,  
    save_total_limit=1,  
    logging_dir="./logs",
    logging_steps=50  
)


# Inicializamos el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Entrenamos el modelo
trainer.train()

# Guardamos el modelo entrenado
output_dir = "./fine_tuned_gpt2_recipes"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

# Generamos receta a partir del titulo
def generar_receta(ingredientes, max_length=20):
    prompt = f"Ingredientes: {ingredientes}\nTítulo:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        inputs, 
        max_length=max_length, 
        num_return_sequences=1, 
        do_sample=True, 
        temperature=0.7
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

C:\Users\Marina\AppData\Local\Temp\ipykernel_17240\1763135968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned_500['texto_entrada'] = "Ingredientes: " + data_cleaned_500['ingredients'].astype(str)
C:\Users\Marina\AppData\Local\Temp\ipykernel_17240\1763135968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned_500['texto_salida'] = "Título: " + data_cleaned_500['title'].astype(str)
C:\Users\Marina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCach

{'loss': 2.0788, 'grad_norm': 3.2403786182403564, 'learning_rate': 4.140893470790378e-05, 'epoch': 0.17}


 34%|███▍      | 100/291 [1:08:19<2:23:18, 45.02s/it]

{'loss': 1.8471, 'grad_norm': 3.0511066913604736, 'learning_rate': 3.2817869415807564e-05, 'epoch': 0.34}


 52%|█████▏    | 150/291 [1:52:30<2:32:25, 64.86s/it] 

{'loss': 1.7796, 'grad_norm': 3.520819664001465, 'learning_rate': 2.422680412371134e-05, 'epoch': 0.52}


 55%|█████▍    | 160/291 [2:01:01<1:46:36, 48.83s/it]

In [ ]:
ingredientes = " "

resultado = generar_receta(ingredientes)
print(resultado)